In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Libs

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.image as mpimg 
from keras.preprocessing import image
from keras import models
import matplotlib.pyplot as plt
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.metrics import classification_report, log_loss, accuracy_score
from tensorflow.keras import layers

In [3]:

train_dir = "/content/drive/MyDrive/spiral2/training"
test_dir  = "/content/drive/MyDrive/spiral2/testing"

IMG_WIDTH, IMG_HEIGHT = (300, 300)
EPOCHS = 50
BATCH_SIZE= 16
CLASSES_NO = 2

## Load dataset

In [4]:
train_datagen_aug = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.1
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

train = train_datagen_aug.flow_from_directory(
    train_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True
)

test = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
)

Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [ ]:
model = keras.Sequential([
        layers.Conv2D(64, (3,3) , activation='relu'),
        layers.Conv2D(64, (3,3)  , activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)), 
        layers.Conv2D(32, (3,3)  , activation='relu'),
        layers.Conv2D(32, (3,3)  , activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dense(activation ="relu", units=128),
        layers.Dense(activation ="relu", units=128),
        layers.Dense(activation ="relu", units=64),
        layers.Dense(activation ="relu", units=64),
        layers.Dense(activation="sigmoid", units=1)
])

model.compile(loss="binary_crossentropy",  optimizer="adam", metrics=['accuracy'])
hist = model.fit(
        train,
        epochs=10,
        validation_data=test,
        )


In [ ]:
history_dict = hist.history
print(history_dict.keys())
import matplotlib.pyplot as plt
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'], color='green')
plt.plot(hist.history['val_loss'], color='red')
plt.title('Model Accuracy')
plt.show()

In [ ]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(
    include_top=False,
    input_shape=(IMG_WIDTH, IMG_HEIGHT, 3),
    weights='imagenet'
)

In [ ]:
new_model = keras.Sequential()
new_model.add(conv_base)
new_model.add(keras.layers.Flatten())
new_model.add(keras.layers.Dense(128, activation="relu"))
new_model.add(keras.layers.Dense(2, activation='softmax'))

conv_base.trainable = False

new_model.summary()

In [ ]:
new_model.compile(
    optimizer=RMSprop(lr=2e-5),
    loss='binary_crossentropy',
    metrics=['acc']
)

In [ ]:
new_hist = new_model.fit(
    train,
    epochs=20,
    batch_size=BATCH_SIZE,
    validation_data=test
)


In [ ]:
acc = new_hist.history['acc']
val_acc = new_hist.history['val_acc']
loss = new_hist.history['loss']
val_loss = new_hist.history['val_loss']

plt.plot(acc, label="Accuracy")
plt.plot(val_acc, label="Validation Accuracy")
plt.legend()
plt.show()


plt.plot(loss, label="Loss")
plt.plot(val_loss, label="Validation Loss")
plt.legend()
plt.show()